# Inversion of Size-Resolved CCN Data 

<div class="alert alert-warning">

Code is based on Session 4 in the tutorial. This script demonstrates how to use the code in a standalone program.
    
</div>


In [ ]:
using DifferentialMobilityAnalyzers, CSV, Gadfly, DataFrames, Printf, LsqFit, SpecialFunctions

# Load a simple comma delimited text file
df = CSV.read("example_data.csv")

# Setup the DMA
t, p, lpm = 293.15, 940e2, 1.666e-5      # Temperature [K], Pressure [Pa], L min-1 to m3 s-1
r₁, r₂, l = 9.37e-3,1.961e-2,0.44369     # DMA geometry [m]
Λ = DMAconfig(t,p,1lpm,4lpm,r₁,r₂,l,0.0,:+,6,:cylindrical)  
δ = setupDMA(Λ, vtoz(Λ,10000), vtoz(Λ,10), 120)

# Interpolate the data onto the DMA grid
𝕣ᶜⁿ = (df,:Dp,:Rcn,δ) |> interpolateDataFrameOntoδ       # CN response distribution
𝕣ᶜᶜⁿ = (df,:Dp,:Rccn,δ) |> interpolateDataFrameOntoδ     # CCN response distribution

# Threshold the data to avoid InF and NaN
function threshold!(𝕟::SizeDistribution, c::Float64, n1::Float64, n2::Float64)
   N = 𝕟.N  
   S = 𝕟.S
   S[(N .<= c) .& (𝕟.Dp .> 150)] .= n2
   N[(N .<= c) .& (𝕟.Dp .> 150)] .= n2
   S[(N .<= c) .& (𝕟.Dp .< 150)] .= n1
   N[(N .<= c) .& (𝕟.Dp .< 150)] .= n1
   𝕟.N = N
end

threshold!(𝕣ᶜⁿ, 0.1, 0.1, 0.1)
threshold!(𝕣ᶜᶜⁿ, 0.1, 0.0, 0.1)

# Compute the activated fraction
𝕒𝕗 = 𝕣ᶜᶜⁿ/𝕣ᶜⁿ

# Activated fraction model
Taf(𝕟,μ,σ) = @. 0.5 * (1.0 + erf((𝕟.Dp - μ)./(sqrt(2.0σ))))

# Multiply charged activation model
𝐈, 𝐒, 𝐀, λ =  δ.𝐈, δ.𝐒, δ.𝐀, 0.5
𝕟ᶜⁿ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣ᶜⁿ  + λ^2 * 𝐒^(-1)*𝕣ᶜⁿ)
model(x,p) = (𝐀 * (𝕟ᶜⁿ.N .* Taf(𝕟ᶜⁿ, p[1], p[2])))./(𝐀 * 𝕟ᶜⁿ.N)

# Fit the data with an initial guess
fit = curve_fit(model, 𝕒𝕗.Dp, 𝕒𝕗.N, [65.0, 3.0])
Ax = fit.param

# Compute the activated fraction model
afmodel = model(δ.Dp, Ax);

## Plot the  data

In [ ]:
df1 = DataFrame(Dp = 𝕒𝕗.Dp, S = 𝕒𝕗.N, Dist = ["𝕒𝕗 (data)" for i = 1:length(𝕒𝕗.Dp)])
df2 = DataFrame(Dp = 𝕒𝕗.Dp, S = afmodel, Dist = ["𝕒𝕗 (model)" for i = 1:length(𝕒𝕗.Dp)])
df = [df1; df2]

xlabels = log10.([10, 100, 500])
plot(
    df,
    x = :Dp,
    y = :S,
    color = :Dist,
    Geom.step,
    Guide.xlabel("Apparent +1 Mobility Diameter (nm)"),
    Guide.ylabel("Fraction (-)"),
    Guide.title("Activated Fraction"),
    Guide.colorkey(; title = ""),
    Guide.xticks(ticks = log10.([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500])),
    Guide.yticks(ticks = collect(0:0.2:1.2)),
    Scale.x_log10(labels = x -> x in xlabels ? @sprintf("%2i", exp10(x)) : ""),
    Scale.color_discrete_manual(["black","darkgoldenrod"]...),
    Coord.cartesian(xmin = log10(10), xmax = log10(500), ymin = 0, ymax = 1.2),
)